In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

dataset_folder = '/drive/Colab Notebooks/Dataset/Train'
valid_folder = '/drive/Colab Notebooks/Dataset/Valid'
test_folder = '/drive/Colab Notebooks/Dataset/Test'

In [ ]:
# Functie pentru a incarca imaginile dintr-un folder si a atribui etichete
def load_images_from_folder(folder, target_shape=(300, 300)):
    images = []
    labels = []
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        if os.path.isdir(subfolder_path):
            for filename in os.listdir(subfolder_path):
                if filename.endswith(('.jpeg', '.jpg', '.png')):
                    img = cv2.imread(os.path.join(subfolder_path, filename))
                    if img is not None:
                        if target_shape is not None:
                            img = cv2.resize(img, target_shape)
                        images.append(img)
                        labels.append(subfolder)
                    else:
                        print(f"Warning: Unable to load {filename}")
    return images, labels

# Functie pentru a extrage caracteristici ale histogramei de culoare
def extract_color_histogram(images, bins=8):
    features = []
    for img in images:
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1, 2], None, [bins, bins, bins], [0, 180, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.append(hist)
    return np.array(features)

# Functie pentru a extrage caracteristici de textura
def extract_lbp_features(images, radius=1, n_points=8):
    features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        hist = hist.astype('float')
        hist /= (hist.sum() + 1e-6)
        features.append(hist)
    return np.array(features)

# Functie pentru a combina toate caracteristicile
def combine_features(images):
    color_histogram = extract_color_histogram(images)
    lbp_features = extract_lbp_features(images)
    return np.hstack([color_histogram, lbp_features])

In [ ]:
# Functie pentru afisat imaginile clasificate
def display_images_with_labels(images, true_labels, predicted_labels, class_mapping, etapa):
    for i, (image, true_label, pred_label) in enumerate(zip(images, true_labels, predicted_labels)):
        plt.figure(figsize=(4, 4))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(f"Real: {class_mapping[true_label]} | Prezis: {class_mapping[pred_label]}")
        plt.axis('off')
        plt.show()

# Functie pentru salvat rezultate
def save_results(etapa, y_true, y_pred, class_mapping):
    results_df = pd.DataFrame({'Clasa Reală': [class_mapping[label] for label in y_true],
                               'Clasa Prezisă': [class_mapping[label] for label in y_pred]})
    results_folder = '/drive/Colab Notebooks/Dataset/Results_KNN'
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)
    results_file = os.path.join(results_folder, f'{etapa}_results.csv')
    results_df.to_csv(results_file, index=False)
    print(f'Rezultate pentru {etapa} la {results_file}')

# Functie pentru salvat imaginile clasificate
def save_classified_images(X_data, y_true, y_pred, class_mapping, etapa):
    results_folder = f'/drive/Colab Notebooks/Dataset/Results_KNN/Images_{etapa}'
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)
    for idx, (true_label, pred_label) in enumerate(zip(y_true, y_pred)):
        image = X_data[idx]
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(f'Clasa Reală: {class_mapping[true_label]}\n Clasa Prezisă: {class_mapping[pred_label]}')
        plt.savefig(os.path.join(results_folder, f'{etapa}_image_{idx}.png'))
        plt.close()

# Functie de predictie si evaluare
def evaluate_model(X_data, y_true, etapa):
    y_pred = knn_model.predict(X_data)
    accuracy = accuracy_score(y_true, y_pred)
    print(f"{etapa} Acuratețe: {accuracy:.4f}")
    print(f"{etapa} Raport de Clasificare:")
    print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title(f'{etapa} Matrice de Confuzie')
    plt.xlabel('Prezis')
    plt.ylabel('Real')
    plt.show()

    display_images_with_labels(valid_images if etapa == 'Validare' else test_images, y_true, y_pred, {idx: label for idx, label in enumerate(label_encoder.classes_)}, etapa)
    save_results(etapa, y_true, y_pred, {idx: label for idx, label in enumerate(label_encoder.classes_)})
    save_classified_images(valid_images if etapa == 'Validare' else test_images, y_true, y_pred, {idx: label for idx, label in enumerate(label_encoder.classes_)}, etapa.lower())

In [ ]:
# Incarca si proceseaza datele de antrenare
dataset_images, dataset_labels = load_images_from_folder(dataset_folder)

X_train = combine_features(dataset_images)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(dataset_labels)

# Scaleaza
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Antreneaza cu k ul dorit (n_neighbors)
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)

# Predictie pe datele de antrenament
y_train_pred = knn_model.predict(X_train)

# Calculeaza acuratetea pe datele de antrenare
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Acuratete Antrenare:", train_accuracy)

Acuratețe Antrenare: 0.8321428571428572


In [ ]:
# Date de validare
valid_images, valid_labels = load_images_from_folder(valid_folder)
X_valid = combine_features(valid_images)
X_valid = scaler.transform(X_valid)
y_valid = label_encoder.transform(valid_labels)
evaluate_model(X_valid, y_valid, "Validare")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Date de testare
test_images, test_labels = load_images_from_folder(test_folder)
X_test = combine_features(test_images)
X_test = scaler.transform(X_test)
y_test = label_encoder.transform(test_labels)
evaluate_model(X_test, y_test, "Testare")


Output hidden; open in https://colab.research.google.com to view.